In [6]:
%%capture
import pandas as pd
import numpy as np
import pyemd
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import nltk
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from nltk import tokenize    
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')
    
def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    
    patterns = [word_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token.lower())
        words.extend(tokens)
    return words

### carregando modelo pré-treinado e nomalizando-o
model = KeyedVectors.load_word2vec_format("wiki_vectors_format_without_stopwords.bin", binary=False)
model.init_sims(replace=True) 

def headline_embeddings(headline):
    emb = []
    for word in headline:
        emb.append(model[word])
    return emb
        
headline = "O que Bolsonaro e Moro colocam em jogo"
headline = parse(headline)
hl_e1 = headline_embeddings(headline)

In [11]:
print("Embedding 10 first for the first word:", hl_e1[0][:10])
print("Embedding lenght:", len(hl_e1[0]))

Embedding 10 first for the first word: [-0.06797549  0.0969245  -0.0308466  -0.07459277 -0.01058708 -0.04934659
  0.08344505 -0.08326908  0.09534609  0.02884346]
Embedding lenght: 300


#### Acima temos um exemplo dos 10 primeiros vetores para a primeira palavra analisada e o tamanho total do embedding para a headline.

In [4]:
headline2 = "Os prejuizos da Operacao Lava Jato para o Brasil"
headline2 = parse(headline2)
hl_e2 = headline_embeddings(headline2)

from math import inf
from math import sqrt

def calc_wmd(vector1, vector2):
    wmd = 0     
    for w in vector1:
        min_dist = inf
        for w2 in vector2:
            min_dist = min(min_dist, sqrt(sum((w - w2) ** 2)))
        wmd += min_dist * 1.0/len(vector1) ### normaliza score pelas palavras movidas        
    return wmd

calc_wmd(hl_e1, hl_e2)
model.wmdistance(headline, headline2)

1.3747184760019384
1.3814151057415365


#### Vemos acima que o método implementado se aproxima ao método wmdistance que é implementado pela biblioteca gensim. 

### 3. Implemente uma função que possa receber qualquer notícia como entrada e retornar as top-3 notícias mais similares (menos distantes) a ela (35 pts).

In [29]:
docs = ["Selecao brasileira joga proxima terca-feira",
        "Neymar é acusado de estupro",
        "Mitos e verdades sobre o consumo de pimenta para a saude",
        "Saiba qual é o tecido que os gatos mais gostam de arranhar",
        "Lava Jato desconfiava de Leo Pinheiro ate versao do triplex do Guaruja",
        "China e EUA concordam em retomar negociacoes comerciais",
        "As gambiarras que todo jogador ja fez na vida",
        "O seda em fim de ciclo com vendas absurdas",
        "Os prejuizos da Operacao Lava Jato para o Brasil",
        "O que Bolsonaro e Moro colocam em jogo"]

query = "Bolsonaro e Trump reforcam lacos diplomaticos"

from gensim.similarities import WmdSimilarity

def topN_sim_docs(query, docs, N):
    top = []
    num_best = N
    processed_docs = [parse(w) for w in docs]
    instance = WmdSimilarity(processed_docs, model, num_best)
    
    p_query = parse(query)
    rank = instance[p_query]

    for i in range(num_best):
        top.append('%d - Similaridade = %.4f - %s' % (i+1,rank[i][1], docs[rank[i][0]]))
        
    return top

print(query)
topN_sim_docs(query, docs, N=3)

Bolsonaro e Trump reforcam lacos diplomaticos


['1 - Similaridade = 0.4720 - O que Bolsonaro e Moro colocam em jogo',
 '2 - Similaridade = 0.4243 - Mitos e verdades sobre o consumo de pimenta para a saude',
 '3 - Similaridade = 0.4236 - Lava Jato desconfiava de Leo Pinheiro ate versao do triplex do Guaruja']

#### Referencias 
[1]https://docs.google.com/presentation/d/1CTmpRHA2gurNM6APTpXzmpfVSZLgRoOszA3XQ-PyCto/edit#slide=id.gc6f73a04f_0_0
[2]https://markroxor.github.io/gensim/static/notebooks/WMD_tutorial.html